## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
from allennlp.data.tokenizers import Token, Tokenizer, PretrainedTransformerTokenizer

import nltk
import numpy as np
from os import listdir
from os.path import join as pathjoin
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import tqdm

from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig
# make deterministic
from mingpt.utils import sample, set_seed
set_seed(42)
np.random.seed(42)

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
transformer_model = 'bert-base-cased'

In [3]:
import math
from torch.utils.data import Dataset


def detokenize(tokens):
    return ' '.join([str(x) for x in tokens[1:-1]]).replace(' ##', '')

class BPEDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [4]:
block_size = 128
tokenizer = PretrainedTransformerTokenizer(transformer_model)

In [5]:
topics = {
    'music': ['music', 'band', 'album', 'songs', 'sound', 'love', 'rock', 'playing', 'guitar', 'jazz'],
    'education': ['student', 'learning', 'courses', 'teaching', 'skills', 'education', 'study', 'college'],
    'politics': ['election', 'votes', 'assembly', 'candidate', 'democratic', 
                 'council', 'minister', 'parliament', 'politician', 'legislative', 'seats', 'vote'],
    'sport': ['football', 'coach', 'basketball', 'tournament', 'schedule', 'games', 'league', 'division', 'team', 'teams'],
    'business': ['technology', 'platform', 'companies', 'industry', 'product', 'design',
                 'upgrade', 'automation', 'users', 'ideas'],
    'literature': ['books', 'literature', 'fiction', 'tolstoy', 'shakespeare', 'dostoevsky', 'romanism', 'fanfics', 'adventure'],
    'crime': ['police', 'court', 'officer', 'incident', 'charges', 'crime', 'prison', 'investigation', 'accused', 'victim'],
    'travel': ['tourist', 'tourism', 'travel', 'seaside', 'vacation', 'beach', 'hotel', 'hostel'],
    'games': ['games', 'xbox', 'players', 'steam', 'cards', 'player', 'damage', 'switch', 'dragon', 'character', 
              'reload', 'console'],
    'arhitecture': ['building', 'historic', 'church', 'buildings', 'brick', 
                    'roof', 'street', 'style', 'tower', 'designed', 'stone', 'architecture']
}

In [6]:
from mingpt.utils import sample

def generate_topic_dataset(train_text_file, state_dict_file, n_layer=8, n_head=8, n_embd=512,
                     texts_count=10, text_len=500):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent)[1:-1] for sent in text_sentences])
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    tokens_set = set(train_dataset.stoi.keys())
    print("dataset is loaded")
    
    mconf = GPTConfig(
        train_dataset.vocab_size, train_dataset.block_size,
        n_layer=n_layer, n_head=n_head, n_embd=n_embd
    )
    model = GPT(mconf)
    model.load_state_dict(torch.load(state_dict_file))
    print("model is loaded")
    
    tconf = TrainerConfig(num_workers=1)
    trainer = Trainer(model, train_dataset, None, tconf)
    
    for topic, topic_keywords in topics.items():
        first_word_candidates = list(set(topic_keywords) & tokens_set)
        
        for text_id in range(texts_count):
            context = [np.random.choice(first_word_candidates)]
            x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
            y = sample(model, x, text_len, temperature=1.0, sample=True, top_k=10)[0]
            completion = ' '.join([train_dataset.itos[int(i)] for i in y]).replace(' ##', '')
            yield completion, topic
        

def test_keywords(train_text_file, state_dict_file, n_layer=8, n_head=8, n_embd=512):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent)[1:-1] for sent in text_sentences])
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    print("dataset is loaded")
    tokens_set = set(train_dataset.stoi.keys())
    for topic, topic_keywords in topics.items():
        print(len(set(topic_keywords) & tokens_set))

In [7]:
GENRE_DATA_DIR = '/home/mlepekhin/data/genre'
GPT_MODELS_DIR = '/home/mlepekhin/models/mini_gpt_bpe/'
LANG = 'en'

In [8]:
'''for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):
    label = train_text_file[:-4]
    print(label)
    test_keywords(
        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),
        pathjoin(GPT_MODELS_DIR, LANG, label)
    )'''

'for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):\n    label = train_text_file[:-4]\n    print(label)\n    test_keywords(\n        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),\n        pathjoin(GPT_MODELS_DIR, LANG, label)\n    )'

In [9]:
result_df = pd.DataFrame()

In [10]:
for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):
    label = train_text_file[:-4]
    print(label)
    for text, topic in generate_topic_dataset(
        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),
        pathjoin(GPT_MODELS_DIR, LANG, label)
    ):
        result_df = result_df.append({'text': text, 'target': label, 'topic': topic}, ignore_index=True)

  0%|          | 0/11 [00:00<?, ?it/s]

A4
data has 191478 characters, 11294 unique.
dataset is loaded
model is loaded


  9%|▉         | 1/11 [05:21<53:38, 321.84s/it]

A7
data has 147526 characters, 9734 unique.
dataset is loaded
model is loaded


 27%|██▋       | 3/11 [16:18<43:25, 325.69s/it]

...

 82%|████████▏ | 9/11 [49:18<11:14, 337.08s/it]

A22
data has 218385 characters, 16623 unique.
dataset is loaded
model is loaded


 91%|█████████ | 10/11 [55:25<05:46, 346.08s/it]

A16
data has 255033 characters, 16025 unique.
dataset is loaded
model is loaded


100%|██████████| 11/11 [1:01:31<00:00, 335.63s/it]


In [13]:
result_df.head()

,target,text,topic
0,A4,love ; you will learn from me by luck ; you co...,music
1,A4,"playing at home with his shoulders . "" You are...",music
2,A4,rock to put and shook her handkerchief away in...,music
3,A4,love with him – and he began to think that thi...,music
4,A4,"songs . In degrees others . Their armour , daz...",music


In [14]:
result_df.to_csv('/home/mlepekhin/data/min_gpt_bpe/en_topic')